In [1]:
import numpy as np
import tensorflow as tf
import random as rn
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
rn.seed(1)
from keras import backend as K
#tf.compat.v1.set_random_seed(1)
#sess = tf.Session(graph=tf.get_default_graph())
#K.set_session(sess)
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.layers import Conv1D,MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import RMSprop
import keras.regularizers
import scipy
import math
import sys
import pandas as pd
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics import mean_squared_error
from scipy.stats import linregress
from scipy import interpolate
from scipy import signal
import pickle
from video_process_utils import *
import collections

C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [2]:
target_col = "GDI"

In [3]:
#assign train/validation/test ids
alldata_processed =\
    pd.read_csv("./data/processed/alldata_processed_with_dev_residual.csv" )
alldata_processed['videoid'] = alldata_processed['videoid'].apply(lambda x: int(x))
alldata_processed = alldata_processed[alldata_processed[target_col].notnull()]
alldata_processed = alldata_processed.groupby(['videoid','side']).head(1)
ids_nonmissing_target = set(alldata_processed['videoid'].unique())

In [4]:
datasplit_df = pd.read_csv('./data/processed/train_test_valid_id_split.csv')
datasplit_df['videoid'] = datasplit_df['videoid'].apply(lambda x: int(x))
all_ids = set(datasplit_df['videoid']).intersection(ids_nonmissing_target)
train_ids = set(datasplit_df[datasplit_df['dataset'] == 'train']['videoid']).intersection(ids_nonmissing_target)
validation_ids = set(datasplit_df[datasplit_df['dataset'] == 'validation']['videoid']).intersection(ids_nonmissing_target)
test_ids = set(datasplit_df[datasplit_df['dataset'] == 'test']['videoid']).intersection(ids_nonmissing_target)

In [5]:
with open('./data/processed/all_processed_video_segments.pickle', 'rb') as handle:
    processed_video_segments = pickle.load(handle)

In [6]:
x_columns = [2*LANK,2*LANK+1,2*LKNE,2*LKNE+1,
        2*LHIP,2*LHIP+1,2*LBTO,2*LBTO+1,
                  2*RANK,2*RANK+1,2*RKNE,2*RKNE+1,
        2*RHIP,2*RHIP+1,2*RBTO,2*RBTO+1,50,51,52,53,54,55,56]

target_dict = {}
for i in range(len(alldata_processed)):
    row = alldata_processed.iloc[i]
    target_dict[row['videoid']] = row[target_col]

In [7]:
X = [t[2] for t in processed_video_segments if t[0] in all_ids]
X = np.stack(X)[:,:,x_columns]

In [8]:
X_train = [t[2] for t in processed_video_segments if t[0] in train_ids]
X_train = np.stack(X_train)[:,:,x_columns]

In [9]:
X_validation = [t[2] for t in processed_video_segments if t[0] in validation_ids]
X_validation = np.stack(X_validation)[:,:,x_columns]

In [10]:
y = np.array([target_dict[t[0]] for t in processed_video_segments if t[0] in all_ids])

In [11]:
y_train = np.array([target_dict[t[0]] for t in processed_video_segments if t[0] in train_ids])

In [12]:
y_validation = np.array([target_dict[t[0]] for t in processed_video_segments if t[0] in validation_ids])

In [13]:
videoid_count_dict = collections.Counter(np.array([t[0] for t in processed_video_segments]))

In [14]:
train_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in train_ids]
train_videoid_weights = np.array(train_videoid_weights).reshape(-1,1)
validation_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in validation_ids]
validation_videoid_weights = np.array(validation_videoid_weights).reshape(-1,1)

In [15]:
target_min = np.min(y_train,axis=0)
target_range = np.max(y_train,axis=0) - np.min(y_train,axis=0)

In [16]:
y_train_scaled = ((y_train-target_min)/target_range).reshape(-1,1)
y_validation_scaled = ((y_validation-target_min)/target_range).reshape(-1,1)

y_validation_scaled = np.hstack([y_validation_scaled,validation_videoid_weights])
y_train_scaled = np.hstack([y_train_scaled,train_videoid_weights])
c_i_factor = np.mean(np.vstack([train_videoid_weights,validation_videoid_weights]))

In [17]:
vid_length = 124

In [18]:
def step_decay(initial_lrate,epochs_drop,drop_factor):
    def step_decay_fcn(epoch):
        return initial_lrate * math.pow(drop_factor, math.floor((1+epoch)/epochs_drop))
    return step_decay_fcn

epochs_drop,drop_factor = (10,0.8)
initial_lrate = 0.001
dropout_amount = 0.5
last_layer_dim = 10
filter_length = 8
conv_dim = 32
l2_lambda = 10**(-3.5)

def w_mse(weights):
    def loss(y_true, y_pred):
        #multiply by len(weights) to make the magnitude invariant to number of components in target
        return K.mean(K.sum(K.square(y_true-y_pred)*weights,axis=1)*tf.reshape(y_true[:,-1],(-1,1)))/c_i_factor
    return loss

#we don't want to optimize for the column counting video occurences of course, but
#they are included in the target so we can use that column for the loss function
weights = [1.0,0]
normal_weights = [1.0,0]


#normalize weights to sum to 1 to prevent affecting loss function
weights = weights/np.sum(weights)
normal_weights = normal_weights/np.sum(normal_weights)

#fixed epoch budget of 100 that empirically seems to be sufficient 
n_epochs = 100

mse_opt = w_mse(weights)

#monitor our actual objective
mse_metric = w_mse(target_range**2*normal_weights)

hyper_str = "params_"
for param in [initial_lrate,epochs_drop,drop_factor,dropout_amount,conv_dim,last_layer_dim,filter_length,l2_lambda]:
    hyper_str = hyper_str + str(param) + "_"

K.clear_session()
#K.set_session(sess)

model = Sequential()
model.add(Conv1D(conv_dim,filter_length, input_dim=X_train.shape[2],input_length=vid_length,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(dropout_amount))
model.add(Flatten())
model.add(Dense(last_layer_dim,activation='relu'))
model.add(Dense(2, activation='linear'))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 8, padding="same", input_shape=(124, 23))`


In [19]:
checkpoint_folder = "./data/checkpoints/cnn_single_checkpoints_%s" % (target_col)

In [20]:
from keras.callbacks import LearningRateScheduler

from keras.callbacks import ModelCheckpoint
from keras.callbacks import TerminateOnNaN

train_model = True

if not os.path.exists(checkpoint_folder):
    os.makedirs(checkpoint_folder)

filepath=checkpoint_folder+"/weights-{epoch:02d}-{val_loss_1:.4f}.hdf5"
if train_model:

    opt = RMSprop(lr=0.0,rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(loss=mse_opt,metrics=[mse_metric],
                  optimizer=opt)

    checkpoint = \
        ModelCheckpoint(filepath, monitor='val_loss_2', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    lr = LearningRateScheduler(step_decay(initial_lrate,epochs_drop,drop_factor))

    history = model.fit(X_train, y_train_scaled,callbacks=[checkpoint,lr,TerminateOnNaN()],
              validation_data=(X_validation,y_validation_scaled),
              batch_size=32, epochs=n_epochs,shuffle=True)

Train on 20426 samples, validate on 2469 samples
Epoch 1/100
20426/20426 [==============================] - 201s 10ms/step - loss: 0.1569 - loss_1: 706.7166 - val_loss: 0.0516 - val_loss_1: 132.0336

Epoch 00001: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-01-132.0336.hdf5
Epoch 2/100
20426/20426 [==============================] - 121s 6ms/step - loss: 0.0368 - loss_1: 111.6967 - val_loss: 0.0284 - val_loss_1: 117.7563

Epoch 00002: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-02-117.7563.hdf5
Epoch 3/100
20426/20426 [==============================] - 116s 6ms/step - loss: 0.0222 - loss_1: 102.3422 - val_loss: 0.0226 - val_loss_1: 116.2315

Epoch 00003: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-03-116.2315.hdf5
Epoch 4/100
20426/20426 [==============================] - 113s 6ms/step - loss: 0.0185 - loss_1: 96.8006 - val_loss: 0.0151 - val_loss_1: 79.7602

Epoch 00004: saving model to ./data/checkpoints/cnn_

Epoch 34/100
20426/20426 [==============================] - 124s 6ms/step - loss: 0.0124 - loss_1: 70.5237 - val_loss: 0.0124 - val_loss_1: 70.6543

Epoch 00034: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-34-70.6543.hdf5
Epoch 35/100
20426/20426 [==============================] - 123s 6ms/step - loss: 0.0123 - loss_1: 69.9313 - val_loss: 0.0121 - val_loss_1: 69.0072

Epoch 00035: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-35-69.0072.hdf5
Epoch 36/100
20426/20426 [==============================] - 106s 5ms/step - loss: 0.0122 - loss_1: 69.3137 - val_loss: 0.0123 - val_loss_1: 69.74050122 - loss_1: 69

Epoch 00036: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-36-69.7405.hdf5
Epoch 37/100
20426/20426 [==============================] - 117s 6ms/step - loss: 0.0122 - loss_1: 69.1385 - val_loss: 0.0128 - val_loss_1: 72.7725

Epoch 00037: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-37-72.7

20426/20426 [==============================] - 67s 3ms/step - loss: 0.0109 - loss_1: 62.4154 - val_loss: 0.0126 - val_loss_1: 72.4632

Epoch 00067: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-67-72.4632.hdf5
Epoch 68/100
20426/20426 [==============================] - 60s 3ms/step - loss: 0.0109 - loss_1: 62.5374 - val_loss: 0.0114 - val_loss_1: 65.2357

Epoch 00068: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-68-65.2357.hdf5
Epoch 69/100
20426/20426 [==============================] - 68s 3ms/step - loss: 0.0107 - loss_1: 61.4399 - val_loss: 0.0122 - val_loss_1: 70.1807

Epoch 00069: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-69-70.1807.hdf5
Epoch 70/100
20426/20426 [==============================] - 86s 4ms/step - loss: 0.0106 - loss_1: 60.3239 - val_loss: 0.0114 - val_loss_1: 65.1483

Epoch 00070: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-70-65.1483.hdf5
Epoch 71/100
20426/20426 

20426/20426 [==============================] - 67s 3ms/step - loss: 0.0099 - loss_1: 56.9438 - val_loss: 0.0118 - val_loss_1: 68.4133

Epoch 00100: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-100-68.4133.hdf5


In [21]:
import statsmodels.api as sm

In [22]:
def undo_scaling(y,target_range,target_min):
    return y*target_range+target_min

In [23]:
weight_files = os.listdir(checkpoint_folder)
weight_files_df = pd.DataFrame(weight_files,columns=['filename'])
weight_files_df['num'] = weight_files_df['filename'].apply(lambda x: int(x.split('-')[1]))
weight_files_df.sort_values(by='num',ascending=True,inplace=True)

In [24]:
def predict_and_aggregate_singlevar(y,X,ids,model,target_col):
    df = pd.DataFrame(y,columns=[target_col])
    target_col_pred = target_col + "_pred"
    videoids = [t[0] for t in processed_video_segments if t[0] in ids]
    df["videoid"] = np.array(videoids)
    preds = model.predict(X)
    df[target_col_pred] = undo_scaling(preds[:,0],target_range,target_min)
    df["count"] = 1
    df = df.groupby(['videoid'],as_index=False).agg({target_col_pred:np.mean,'count':np.sum,target_col:np.mean})
    df['ones'] = 1
    return df

In [25]:
video_ids = [t[0] for t in processed_video_segments if t[0] in all_ids]
predictions_df = pd.DataFrame(video_ids,columns=['videoid'])
predictions_df[target_col] = y
predictions_df = predictions_df.merge(right=datasplit_df[['videoid','dataset']],on=['videoid'],how='left')

In [26]:
for i in range(0,len(weight_files_df)):
    weight_file = weight_files_df['filename'].iloc[i]
    print(weight_file)
    model.load_weights(checkpoint_folder + "/%s" % (weight_file))
    preds = model.predict(X)
    predictions_df["%s_pred_%s" % (target_col,i)] = undo_scaling(preds[:,0],target_range,target_min)

weights-01-132.0336.hdf5
weights-02-117.7563.hdf5
weights-03-116.2315.hdf5
weights-04-79.7602.hdf5
weights-05-82.8264.hdf5
weights-06-82.7892.hdf5
weights-07-93.7875.hdf5
weights-08-75.0590.hdf5
weights-09-72.2348.hdf5
weights-10-81.7964.hdf5
weights-11-80.3592.hdf5
weights-12-85.9120.hdf5
weights-13-91.9541.hdf5
weights-14-70.5512.hdf5
weights-15-74.2539.hdf5
weights-16-87.7894.hdf5
weights-17-72.0960.hdf5
weights-18-79.3520.hdf5
weights-19-72.3368.hdf5
weights-20-68.4154.hdf5
weights-21-70.3173.hdf5
weights-22-76.2482.hdf5
weights-23-73.8993.hdf5
weights-24-77.6342.hdf5
weights-25-69.0302.hdf5
weights-26-69.9709.hdf5
weights-27-76.4819.hdf5
weights-28-74.0666.hdf5
weights-29-96.2329.hdf5
weights-30-65.9987.hdf5
weights-31-67.0506.hdf5
weights-32-68.4950.hdf5
weights-33-67.8335.hdf5
weights-34-70.6543.hdf5
weights-35-69.0072.hdf5
weights-36-69.7405.hdf5
weights-37-72.7725.hdf5
weights-38-70.1361.hdf5
weights-39-69.6951.hdf5
weights-40-66.0261.hdf5
weights-41-66.8292.hdf5
weights-42-68

In [28]:
predictions_df.groupby(['videoid','dataset'],as_index=False).mean().to_csv("./data/predictions/cnn_%s_predictions_all_epochs.csv" % (target_col),index=False)